In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# # data['HeartDisease'].value_counts()
# data.columns
# import seaborn as sns
# # corr=data.corr
# # sns.heatmap(data,data.corrcoef())
# from sklearn.preprocessing import OneHotEncoder
# # data.head()
# # data['ChestPainType'].unique()
# # data_=OneHotEncoder(drop='first').fit(data['ChestPainType','Sex','RestingECG','ST_Slope'])
# df=pd.get_dummies(data, columns = ['ChestPainType','Sex','RestingECG','ST_Slope'])
# # df.head()
# df['ExerciseAngina']=[0 if x=='N' else 1 for x in df['ExerciseAngina'] ]

# # df.head()
# df.columns
# # y=data['HeartDisease']
# # data.columns
# x= df.drop(['HeartDisease'],axis=1)
# x.head()
# import Standardscaler

In [3]:
import pandas as pd
data=pd.read_csv(r'../input/heart-failure-prediction/heart.csv')

In [4]:
data.head()
# data.shape
data.columns

In [5]:
# data.describe()
# data.isna().sum() #No missing value found
# data['HeartDisease'].value_counts() #Balanced class
# data.columns

In [6]:
import seaborn as sns
# correlation_coeff= data.corr()
# # print(correlation_coeff)
# sns.heatmap(data=correlation_coeff,annot=True)

In [7]:
#Check for categorical variables
#Feature Engineering
# data[['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']].nunique()
#Onehot encoding
categorical_cols = ['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']
encoded_data = pd.get_dummies(data, columns = categorical_cols, drop_first= True)

In [8]:
encoded_data.head()
# encoded_data.columns
correlation_coeff_= encoded_data.corr()
# print(correlation_coeff)
import matplotlib.pyplot as plt
plt.figure(figsize = (16,8))
sns.heatmap(data=correlation_coeff_,annot=True)




In [9]:
encoded_data.head()

In [10]:
X,y= encoded_data.drop(['HeartDisease'], axis=1), encoded_data['HeartDisease']

In [11]:
#Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
scaled_feat= scaler.fit_transform(X)

In [12]:
scaled_df= pd.DataFrame(scaled_feat,columns=X.columns)

In [13]:
#Split the data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(scaled_df,y,test_size=0.3)

In [14]:
#Will try Logistic Regression first
from sklearn.linear_model import LogisticRegression
LogRM= LogisticRegression()
#Training started
LogRM.fit(X_train,y_train)

In [15]:
from sklearn.model_selection import GridSearchCV
LogRM_= LogisticRegression()
param = {'C':[0.0001,0.001,0.01,0.1,1,10,100]}
grid = GridSearchCV(LogRM_, param)
grid.fit(X_train,y_train)

In [16]:
print(f"Best Estimator: {grid.best_estimator_}")
print(f"Best Params: {grid.best_params_}")

In [17]:
#Predicting on test data
grid_X_pred= grid.predict(X_test)
X_pred= LogRM.predict(X_test)

In [18]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,X_pred))
print('-------------------')
print(confusion_matrix(y_test,X_pred))

In [19]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,grid_X_pred))
print('-------------------')
print(confusion_matrix(y_test,grid_X_pred))

In [20]:
# report=classification_report(y_test,grid_X_pred, output_dict= True)
# macro_precision =  report['macro avg']['precision'] 
# macro_recall = report['macro avg']['recall']    
# macro_f1 = report['macro avg']['f1-score']
# accuracy= report['accuracy']

# print(macro_precision, macro_recall, macro_f1, accuracy)

In [21]:
#Create a dictionary to apply 3 different models i.e. LR, SVM, KNN
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
model_dict= {'LR':LogisticRegression(),'SVM': SVC(), 'KNN': KNeighborsClassifier()}

In [22]:
# final_dict,report_dict={},{}
# for model_name, model in model_dict.items():
#     clf=model
#     clf.fit(X_train,y_train)
#     pred= clf.predict(X_test)
#     report= classification_report(y_test, pred, output_dict= True)
#     report_dict['macro_precision'] =  report['macro avg']['precision'] 
#     report_dict['macro_recall'] = report['macro avg']['recall']    
#     report_dict['macro_f1'] = report['macro avg']['f1-score']
#     report_dict['accuracy']= report['accuracy']
#     final_dict[model_name]=report_dict
# summary= pd.DataFrame(final_dict)

In [23]:
#Will try SVM now
from sklearn.svm import SVC
svm1= SVC()
#Training started
svm1.fit(X_train,y_train)
pred1= svm1.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test,pred1))
print('-------------------')
print(confusion_matrix(y_test,pred1))

In [24]:
def get_report(model):
    report_dict={}
    clf=model
    clf.fit(X_train,y_train)
    pred= clf.predict(X_test)
    report= classification_report(y_test, pred, output_dict= True)
    report_dict['macro_precision'] =  report['macro avg']['precision'] 
    report_dict['macro_recall'] = report['macro avg']['recall']    
    report_dict['macro_f1'] = report['macro avg']['f1-score']
    report_dict['accuracy']= report['accuracy']
    
    return report_dict

In [25]:
final_dict= {}
for model_name, model in model_dict.items():
    rd= get_report(model)
    final_dict[model_name]=rd
summary= pd.DataFrame(final_dict)

In [26]:
summary